# Initialize

In [1]:
data_dir = '/Users/eyshin/Documents/Research/IN/processed_data/'
result_dir = '/Users/eyshin/Documents/Research/IN/results/'

subj_list = [
    'IN04', 'IN05', 'IN07', 'IN09', 'IN10', 'IN11',
    'IN12', 'IN13', 'IN14', 'IN15', 'IN16', 'IN17',
    'IN18', 'IN23', 'IN24', 'IN25', 'IN26', 'IN28',
    'IN29', 'IN30', 'IN31', 'IN32', 'IN33', 'IN34',
    'IN35', 'IN38', 'IN39', 'IN40', 'IN41', 'IN42',
    'IN43', 'IN45', 'IN46' 
]

num_SUBJ = len(subj_list)

# Statistical Test Header

In [2]:
import numpy as np
import nilearn.image
import nilearn.mass_univariate

In [3]:
# load mask file
mask_path = data_dir + 'full_mask.group33.nii.gz'
mask_img = nilearn.image.load_img(mask_path)
mask = np.logical_not(mask_img.get_data().astype(np.bool))

# GNB classifier results

In [4]:
# load data

scores_each_subject = []

for subj in subj_list:
    score = nilearn.image.load_img(result_dir + '%s_move_r3_gnb.nii.gz' % subj).get_data()
    score_ma = np.ma.array(score, mask=mask)
    scores_each_subject.append(score)
    
scores_each_subject = np.ma.array(scores_each_subject, mask=[mask for i in range(num_SUBJ)])

In [9]:
# show data summary

shape = scores_each_subject[1, :, :, :].shape

for i, subj in enumerate(subj_list):
    data = np.array([d for d in scores_each_subject[i, :, :, :].flat if d is not np.ma.masked])
    mean = np.mean(data)
    std = np.std(data)
    median = np.median(data)
    print('%s: mean %.2f, median %.2f, std %.2f' % (subj, mean, median, std))
    
# if you need to standardization, use this code
#    scores_each_subject[i, :, :, :] = (scores_each_subject[i, :, :, :] - mean) / std

IN04: mean 0.50, median 0.50, std 0.04
IN05: mean 0.49, median 0.49, std 0.04
IN07: mean 0.52, median 0.52, std 0.05
IN09: mean 0.52, median 0.52, std 0.04
IN10: mean 0.52, median 0.52, std 0.03
IN11: mean 0.47, median 0.46, std 0.05
IN12: mean 0.47, median 0.47, std 0.06
IN13: mean 0.50, median 0.50, std 0.05
IN14: mean 0.49, median 0.49, std 0.05
IN15: mean 0.48, median 0.48, std 0.05
IN16: mean 0.52, median 0.53, std 0.06
IN17: mean 0.51, median 0.51, std 0.05
IN18: mean 0.50, median 0.50, std 0.05
IN23: mean 0.50, median 0.50, std 0.04
IN24: mean 0.49, median 0.49, std 0.05
IN25: mean 0.53, median 0.53, std 0.05
IN26: mean 0.47, median 0.47, std 0.05
IN28: mean 0.51, median 0.51, std 0.05
IN29: mean 0.50, median 0.50, std 0.03
IN30: mean 0.50, median 0.50, std 0.05
IN31: mean 0.54, median 0.54, std 0.03
IN32: mean 0.51, median 0.51, std 0.01
IN33: mean 0.52, median 0.52, std 0.05
IN34: mean 0.50, median 0.50, std 0.05
IN35: mean 0.50, median 0.50, std 0.05
IN38: mean 0.51, median 0

In [ ]:
# perform statistical test and save t map

t_img = np.zeros(shape)

for j in range(shape[0]):
    for k in range(shape[1]):
        for l in range(shape[2]):
            # check voxel is in group mask
            if not np.all(scores_each_subject[:, j, k, l].mask):
                # perform permuted OLS
                _, t_score, _ = nilearn.mass_univariate.permuted_ols(
                    np.ones((num_SUBJ, 1)),  # one group
                    scores_each_subject[:, j, k, l].reshape(-1, 1),  # make data (num_subject, data vector)
                    n_perm=5000,
                    two_sided_test=True,
                    n_jobs=4)
                
                # save results as image
                t_img[j, k, l] = t_score

        print('%d, %d, ..., ' % (j, k), end='\r')
        
t_img = nilearn.image.new_img_like(mask_img, t_img)
t_img.to_filename(result_dir + 'tstat_radius3_gnb.nii.gz')

# SVC classifier results

In [ ]:
# load data

scores_each_subject = []

for subj in subj_list:
    score = nilearn.image.load_img(result_dir + '%s_move_r3_svc.nii.gz' % subj).get_data()
    score_ma = np.ma.array(score, mask=mask)
    scores_each_subject.append(score)
    
scores_each_subject = np.ma.array(scores_each_subject, mask=[mask for i in range(num_SUBJ)])

In [ ]:
# show data summary

shape = scores_each_subject[1, :, :, :].shape

for i, subj in enumerate(subj_list):
    data = np.array([d for d in scores_each_subject[i, :, :, :].flat if d is not np.ma.masked])
    mean = np.mean(data)
    std = np.std(data)
    median = np.median(data)
    print('%s: mean %.2f, median %.2f, std %.2f' % (subj, mean, median, std))
    
# if you need to standardization, use this code
#    scores_each_subject[i, :, :, :] = (scores_each_subject[i, :, :, :] - mean) / std

In [ ]:
# perform statistical test and save t map

t_img = np.zeros(shape)

for j in range(shape[0]):
    for k in range(shape[1]):
        for l in range(shape[2]):
            # check voxel is in group mask
            if not np.all(scores_each_subject[:, j, k, l].mask):
                # perform permuted OLS
                _, t_score, _ = nilearn.mass_univariate.permuted_ols(
                    np.ones((num_SUBJ, 1)),  # one group
                    scores_each_subject[:, j, k, l].reshape(-1, 1),  # make data (num_subject, data vector)
                    n_perm=5000,
                    two_sided_test=True,
                    n_jobs=4)
                
                # save results as image
                t_img[j, k, l] = t_score

        print('%d, %d, ..., ' % (j, k), end='\r')
        
t_img = nilearn.image.new_img_like(mask_img, t_img)
t_img.to_filename(result_dir + 'tstat_radius3_svc.nii.gz')